In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.trainers import ChatterBotCorpusTrainer
import os
import wikipediaapi
from flask import Flask, render_template, request
# from chatbot import Chat,reflections,multiFunctionCall
# import wikipedia
import nltk
import random
import string
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from chatterbot.trainers import ListTrainer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicki\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nicki\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
bot=ChatBot("Anurag")
trainer = ChatterBotCorpusTrainer(bot)
# Training 
trainer.train("chatterbot.corpus.english")

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [3]:
#Disease Prediction
# Reading the symptoms dataset
sym_data = pd.read_csv("training.csv")

sym_data = pd.concat([sym_data]*2, ignore_index=True)
cols = sym_data.columns
cols = cols[1:]

x = sym_data[cols]
y = sym_data['Disease']


#Split data into train test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
clf_mnb=mnb.fit(x_train,y_train)

features = cols

feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

input_features = [feature_dict['skin_rash'], feature_dict['fatigue'], feature_dict['chills']]

arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)

arr = np.array(arr).reshape(-1,len(arr))

predicted_disease = (clf_mnb.predict(arr))
print("The disease predicted based on given symptoms is : " + predicted_disease[0])

The disease predicted based on given symptoms is : Dengue


In [4]:
# Making to accept one to many symptoms to predict disease 
input_features = [feature_dict['stomach_bleeding']]

test_input_features = input_features.copy()
arr_test = []
for i in range(len(features)):
    for j in range(len(test_input_features)):
        if i == test_input_features[j]:
            #i = int(i/test_input_features[j])
            i = "True"
    arr_test.append(i)  
    
for i in range(len(arr_test)):
    if arr_test[i] == "True":
        arr_test[i] = 1
    else:
        arr_test[i] = 0
            
    
arr_test = np.array(arr_test).reshape(-1,len(arr_test))

predicted_disease = (clf_mnb.predict(arr_test))
print("The disease predicted based on given symptoms is : " + predicted_disease[0])

The disease predicted based on given symptoms is : Hepatitis E


In [5]:
# taking symptoms as input and predicting the disease
def get_disease(symptom_name):
  symptom_name = symptom_name.replace("I am suffering from ", "")
  symptom_name = symptom_name.split(',')
  input_features = []
  for i in range(len(symptom_name)):
    input_features.append(feature_dict[symptom_name[i]])
  print(input_features)

  arr = []
  for i in range(len(features)):
    for j in range(len(input_features)):
        if i == input_features[j]:
            i = "True"
    arr.append(i)
    
  for i in range(len(arr)):
    if arr[i] =="True":
        arr[i] = 1
    else:
        arr[i] = 0
    

  arr = np.array(arr).reshape(-1,len(arr))
  predictedDisease = (mnb.predict(arr))
  CB_response = predictedDisease[0]
  return CB_response

get_disease("I am suffering from hip_joint_pain,knee_pain,swelling_joints,painful_walking")

[79, 78, 82, 121]


'Osteoarthristis'

In [6]:

def drug_recommender_system(disease_predicted):
  #Reading the dataset with sentiment analysis of reviews
  merged_wavg = pd.read_csv('Drug_Recommender.csv')
  # Taking drugs only with positive reviews for recommendation
  merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Negative"].index)
  merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Neutral"].index)

  # Taking predicted disease as input and recommending drug based on highest weighted average of ratings
  grouped = merged_wavg.groupby('Disease')
  #disease_predicted = disease_predicted.lower()
  recommended_drug = pd.DataFrame(grouped.get_group(disease_predicted).nlargest(4,['Rating_Wavg']))
  recommended_drug.Drug = recommended_drug.Drug.str.upper()
  recommended_drug = recommended_drug.drop(columns= ['Review', 'Review_Sentiment','Rating_Wavg'])

  recDrug = list(recommended_drug["Drug"])
  return recDrug



In [7]:
# taking predicted disease as input and recommending drug based on highest weighted average of ratings
def get_drug(user_disease):
  user_disease = user_disease.replace("Medicine for ", "")
  drugName = drug_recommender_system(user_disease.capitalize())
  print(drugName)
  print(len(drugName))
  if len(drugName) > 2 :
    return "Please take this medicine: " + drugName[0] + ", "+ drugName[1] + ","+drugName[2] + "," + drugName[3]
  elif len(drugName) >= 1 and len(drugName) <=2 or len(drugName) != 0 :
    return "Please take this medicine: " + drugName[0]
  elif len(drugName) == 0:
    return "Please Come With Different Name ."

mesg = "Medicine for Osteoarthristis"
print(get_drug(mesg))
print("Osteoarthritis" == "Osteoarthristis")

['GLUCOSAMINE', 'ZOSTRIX', 'BEXTRA', 'VALDECOXIB']
4
Please take this medicine: GLUCOSAMINE, ZOSTRIX,BEXTRA,VALDECOXIB
False


In [8]:
sent_tokens=[]
def Normalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))
    
    #remove ascii
    new_words = []
    for word in word_token:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    
    #Remove tags
    rmv = []
    for w in new_words:
        text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
        rmv.append(text)
        
    #pos tagging and lemmatization
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    lmtzr = WordNetLemmatizer()
    lemma_list = []
    rmv = [i for i in rmv if i]
    for token, tag in nltk.pos_tag(rmv):
        lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
        lemma_list.append(lemma)
    return lemma_list


welcome_input = ("greetings", "sup", "what's up","hey","how are you")
welcome_response = ["*nods*", "hello", "I am glad! You are talking to me"]
def welcome(user_response):
    for word in user_response.split():
        if word.lower() in welcome_input:
            return random.choice(welcome_response)




In [9]:

def generateResponse(user_response):
    try:
        robo_response=''
        sent_tokens.append(user_response)
        TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
        tfidf = TfidfVec.fit_transform(sent_tokens)
        #vals = cosine_similarity(tfidf[-1], tfidf)
        vals = linear_kernel(tfidf[-1], tfidf)
        idx=vals.argsort()[0]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[0]
         #print("asdfdsf:",user_response)
        #welcome(user_response)
        print(user_response)  
        if "I am suffering from" in user_response:
            print("from 1st condition")
            if user_response:
                predictedDisease = get_disease(user_response)
                print(predictedDisease)
                predictedDrug = drug_recommender_system(predictedDisease)
                return "You are Infected by " + predictedDisease + ". This Medicines " + predictedDrug[0] +  ","+ predictedDrug[1] + "," + predictedDrug[2] + " will treat you soon ."
                         
        elif "Medicine for" in user_response:
            print("from 2nd condition")
            if user_response:
                predictedDrug = get_drug(user_response)
                return predictedDrug 
        else:
            #robo_response = robo_response+sent_tokens[idx]
            #return robo_response
            print("from 3rd condition")
            try:
                robo_response=bot.get_response(user_response)
                print(robo_response)
                return robo_response
            except Exception as e:
                print("Sorry, I do not know the response")
    except:
        #return welcome(user_response)
        return "Sorry, but idk"




In [10]:
mesg = "I am suffering from knee_pain,hip_joint_pain"
mesg1 = "Medicine for fever"
res2 = generateResponse(mesg)

print(res2)

I am suffering from knee_pain,hip_joint_pain
from 1st condition
[78, 79]
Osteoarthristis
You are Infected by Osteoarthristis. This Medicines GLUCOSAMINE,ZOSTRIX,BEXTRA will treat you soon .


In [11]:
%mkdir templates -p

A subdirectory or file templates already exists.
Error occurred while processing: templates.
A subdirectory or file -p already exists.
Error occurred while processing: -p.


In [12]:
%%writefile templates/home.html
<!doctype html>
<html lang="en">

<head>
    <link
      rel="shortcut icon"
      type="image/x-icon"
      href="/content/drive/My Drive/gtdt/Notebooks/UI_Files/Doctor_Icon.jpg"
    />
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
    <style>
      body {
        font-family: monospace;
      }
      h1 {
        background-color: yellow;
        display: inline-block;
        font-size: 3em;
        margin: 0;
        padding: 14px;
      }
      h3 {
        color: black;
        font-size: 20px;
        margin-top: 3px;
        text-align: center;
      }
      #chatbox {
        margin-left: auto;
        margin-right: auto;
        width: 40%;
        margin-top: 60px;
      }
      #userInput {
        margin-left: auto;
        margin-right: auto;
        width: 40%;
        margin-top: 60px;
      }
      #textInput {
        width: 90%;
        border: none;
        border-bottom: 3px solid black;
        font-family: monospace;
        font-size: 17px;
      }
      .userText {
        color: white;
        font-family: monospace;
        font-size: 17px;
        text-align: right;
        line-height: 30px;
      }
      .userText span {
        background-color: #808080;
        padding: 10px;
        border-radius: 2px;
      }
      .botText {
        color: white;
        font-family: monospace;
        font-size: 17px;
        text-align: left;
        line-height: 30px;
      }
      .botText span {
        background-color: #4169e1;
        padding: 10px;
        border-radius: 2px;
      }
      #tidbit {
        position: absolute;
        bottom: 0;
        right: 0;
        width: 300px;
      }
      .boxed {
        margin-left: auto;
        margin-right: auto;
        width: 78%;
        margin-top: 60px;
        border: 1px solid green;
      }
      .box {
        border: 2px solid black;
      }
    </style>
</head>

<body>
  <body>
    <center>
      <h1>
      Drug Recommender System
      </h1>
    </center>
<div class="box"></div>
    <div class="boxed">
      <div>
        <div id="chatbox">
          <img
            src="https://image.freepik.com/free-vector/doctor-icon-avatar-white_136162-58.jpg"
            alt="Navaneethan"
            style="width:40px;height:40px;"
          />
          <p class="botText">
            <span>Hi! I am Navaneethan. I shall be your personal doctor.</span>
          </p>
        </div>
        <div id="userInput">
          <input id="textInput" type="text" name="msg" placeholder="Message" />
        </div>
      </div>
    </div>

    <script>
        function getBotResponse() {
          var rawText = $("#textInput").val();
          var userHtml = '<p class="userText"><span>' + rawText + "</span></p>";
          $("#textInput").val("");
          $("#chatbox").append(userHtml);
          document
            .getElementById("userInput")
            .scrollIntoView({ block: "start", behavior: "smooth" });
          $.get("/get", { msg: rawText }).done(function(data) {
            var botHtml = '<p class="botText"><span>' + data + "</span></p>";
            $("#chatbox").append(botHtml);
            document
              .getElementById("userInput")
              .scrollIntoView({ block: "start", behavior: "smooth" });
          });
        }
        $("#textInput").keypress(function(e) {
          if (e.which == 13) {
            getBotResponse();
          }
        });
</script>
  </body>
</body>

</html>

Overwriting templates/home.html


In [13]:
%%writefile templates/about.html
<!DOCTYPE html>
<html>
<head>
<title>Page Title</title>
</head>
<body>

<h1>This is a Heading</h1>
<p>This is a paragraph.</p>

</body>
</html>

Overwriting templates/about.html


In [14]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
#run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return render_template('chatbot.html')
@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print(type(userText))
    a=str(generateResponse(userText))
    print(a)
    sent_tokens.remove(userText)
    return a
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 09:11:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 09:11:38] "GET /content/drive/My%20Drive/gtdt/Notebooks/UI_Files/Doctor_Icon.jpg HTTP/1.1" 404 -


<class 'str'>
I am suffering from hip_joint_pain,knee_pain,swelling_joints,painful_walking
from 1st condition
[79, 78, 82, 121]
Osteoarthristis


127.0.0.1 - - [25/May/2022 09:11:50] "GET /get?msg=I%20am%20suffering%20from%20hip_joint_pain%2Cknee_pain%2Cswelling_joints%2Cpainful_walking HTTP/1.1" 200 -


You are Infected by Osteoarthristis. This Medicines GLUCOSAMINE,ZOSTRIX,BEXTRA will treat you soon .


...

You are Infected by Osteoarthristis. This Medicines GLUCOSAMINE,ZOSTRIX,BEXTRA will treat you soon .
<class 'str'>
Medicine for Osteoarthristis
from 2nd condition


127.0.0.1 - - [25/May/2022 09:17:57] "GET /get?msg=Medicine%20for%20Osteoarthristis HTTP/1.1" 200 -


['GLUCOSAMINE', 'ZOSTRIX', 'BEXTRA', 'VALDECOXIB']
4
Please take this medicine: GLUCOSAMINE, ZOSTRIX,BEXTRA,VALDECOXIB
<class 'str'>
thanking
from 3rd condition


127.0.0.1 - - [25/May/2022 09:18:10] "GET /get?msg=thanking HTTP/1.1" 200 -


you can't buy much for a dollar anymore.
you can't buy much for a dollar anymore.
